# 개발환경
- pytorch=1.10
- opencv=4.5.4
- numpy=1.21.4

In [1]:
from unet_a2c import *
import cv2
import numpy as np
import os
import torch
from tqdm.notebook import tqdm


## validation dataset 경로 입력
### 한 폴더에 png, npy 파일 같이 배치

In [2]:
VALIDATION_PATH='./test_image/A2C/'
# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE = 'cpu'
W_SIZE = 600
H_SIZE = 400
BATCH_SIZE = 1
output_path = './outputs/A2C/'

## 데이터셋 준비 및 모델 로딩

In [3]:
file_list_val = os.listdir(VALIDATION_PATH)
npy_list_val = list()
png_list_val = list()

for name in file_list_val:
    if name[-4:]=='.png':
        png_list_val.append(name)
    elif name[-4:]=='.npy':
        npy_list_val.append(name)

dataset_val = list()
for name in png_list_val:
    common_name = name[:-4]
    # npy_obj = np.load(VALIDATION_PATH+common_name+'.npy')
    # print(npy_obj)
    # break
    # npy_obj = cv2.resize(npy_obj, (W_SIZE,H_SIZE)).reshape(1,H_SIZE,W_SIZE)*255
    # npy_obj = cv2.resize(npy_obj, (W_SIZE,H_SIZE)).reshape(1,H_SIZE,W_SIZE)*255
    png_obj = cv2.imread(VALIDATION_PATH+common_name+'.png', 0)
    h, w = png_obj.shape
    png_obj = cv2.resize(png_obj, (W_SIZE,H_SIZE)).reshape(1,H_SIZE,W_SIZE)
    # dataset_val.append((png_obj, npy_obj))
    dataset_val.append((png_obj,(h,w, name)))

test_dataset = dataset_val
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
)
BEST_MODEL_PATH = 'best_model_A2C.pth'
unet = DoubleUNet(1,1).to(DEVICE)
unet.load_state_dict(torch.load(BEST_MODEL_PATH))
unet.eval()

DoubleUNet(
  (first_unet): UNet(
    (inc): DoubleConv(
      (double_conv): Sequential(
        (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (down1): Down(
      (maxpool_conv): Sequential(
        (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (1): DoubleConv(
          (double_conv): Sequential(
            (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
            (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       

## validation set predict
### 종합된 DSC , JI 수치 출력

In [4]:

TP_collect = list()
LABEL_TRUE_SUM_collect = list()
PREDICTI_TRUE_SUM_collect = list()
outputs = list()
for img, (h, w, name) in tqdm(test_loader):
    img = img.to(DEVICE)
    # label_test = label.float().to(DEVICE)
                
    outputs_test,  adap_outputs, adap_label = unet(img.float(), img.float())
    tmp = outputs_test.detach().numpy().squeeze().squeeze()
    ret = cv2.resize(tmp, (int(w),int(h)))
    ret = np.round(ret).astype(int)
    cv2.imwrite(output_path+str(name[0]), ret*255)
    outputs.append(ret)


  0%|          | 0/100 [00:00<?, ?it/s]